## Using Google Object Detection API for detecting objects in images

Adapted from Tensorflow's <a herf='https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb'>Object Detection Tutorial</a>

### Import Libraries

In [3]:
#Import libraries..
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image

### Setup Environment

In [4]:
# This is needed to display the images.
%matplotlib inline

### Import Models

The models were downloaded from: <a href="https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md">Tensorflow's Object Detection API</a>  

In [7]:
MODEL_MOBILENET = './models/ssd_mobilenet_v1.pb'
MODEL_INCEPTION = './models/ssd_inception_v2.pb'

### Load a (frozen) model

In [59]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(MODEL_MOBILENET, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

### Load labels

In [95]:
#Import helper function
from utils import create_label_map 

#Set #label and path to labels 
LABELS_PATH = './data/mscoco_label_map.pbtxt'
NUM_CLASSES = 90

#Creating mapping from integer index --> String Name
label_map = create_label_map.load_labelmap(LABELS_PATH)
print (label_map)
print (len(label_map[1]))
for k in label_map.keys():
    label_map[k] = label_map[k].strip('\"')
print (label_map)    
    
#label_map = label_map_util.load_labelmap(LABELS_PATH)
#categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
#category_index = label_map_util.create_category_index(categories)

{1: '"person"', 2: '"bicycle"', 3: '"car"', 4: '"motorcycle"', 5: '"airplane"', 6: '"bus"', 7: '"train"', 8: '"truck"', 9: '"boat"', 10: '"traffic light"', 11: '"fire hydrant"', 13: '"stop sign"', 14: '"parking meter"', 15: '"bench"', 16: '"bird"', 17: '"cat"', 18: '"dog"', 19: '"horse"', 20: '"sheep"', 21: '"cow"', 22: '"elephant"', 23: '"bear"', 24: '"zebra"', 25: '"giraffe"', 27: '"backpack"', 28: '"umbrella"', 31: '"handbag"', 32: '"tie"', 33: '"suitcase"', 34: '"frisbee"', 35: '"skis"', 36: '"snowboard"', 37: '"sports ball"', 38: '"kite"', 39: '"baseball bat"', 40: '"baseball glove"', 41: '"skateboard"', 42: '"surfboard"', 43: '"tennis racket"', 44: '"bottle"', 46: '"wine glass"', 47: '"cup"', 48: '"fork"', 49: '"knife"', 50: '"spoon"', 51: '"bowl"', 52: '"banana"', 53: '"apple"', 54: '"sandwich"', 55: '"orange"', 56: '"broccoli"', 57: '"carrot"', 58: '"hot dog"', 59: '"pizza"', 60: '"donut"', 61: '"cake"', 62: '"chair"', 63: '"couch"', 64: '"potted plant"', 65: '"bed"', 67: '"din

### Helper to load image into numpy array

In [96]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

### Detection

In [97]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.

import os 
from utils import visualization_utils as vis_util


PATH_TO_TEST_IMAGES_DIR = './testImages'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3)]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [98]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path in TEST_IMAGE_PATHS:
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            print (num_detections)
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                label_map,
                use_normalized_coordinates=True,
                line_thickness=8)
             

            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)

Tensor("num_detections:0", shape=(1,), dtype=float32)


TypeError: string indices must be integers